In [12]:
# Install dependencies (run this in the first cell)
!pip install yfinance prophet gradio --quiet


In [13]:
# Import libraries
import yfinance as yf
import pandas as pd
from prophet import Prophet
import gradio as gr
from datetime import datetime
import re

In [14]:
# Initialize data stores
data_store = {}
forecast_store = {}
fig_store = {}

In [15]:
# Fetch fresh data
def refresh_data(symbol):
    print(f"Refreshing data for {symbol}...")
    df = yf.download(symbol, period="5y")
    df = df.reset_index()[['Date', 'Close']]
    df.columns = ['ds', 'y']
    df['ds'] = pd.to_datetime(df['ds']).dt.tz_localize(None)
    print(f"Retrieved {len(df)} records for {symbol}.")
    return df


In [16]:
# Forecast prices
def forecast_prices(df, days=14):
    model = Prophet()
    model.fit(df)
    future = model.make_future_dataframe(periods=days)
    forecast = model.predict(future)
    fig = model.plot(forecast)
    return forecast, fig


In [17]:
# Parse user input
def parse_user_input(user_input):
    user_input = user_input.lower()
    if "refresh" in user_input:
        coin = re.findall(r"(btc|eth)", user_input)
        return "refresh", coin[0].upper() if coin else None, None
    elif "predict" in user_input or "forecast" in user_input:
        coin = re.findall(r"(btc|eth)", user_input)
        days = re.findall(r"(\d+)\s+day", user_input)
        return "predict", coin[0].upper() if coin else None, int(days[0]) if days else 7
    elif "plot" in user_input or "show" in user_input or "chart" in user_input:
        coin = re.findall(r"(btc|eth)", user_input)
        return "plot", coin[0].upper() if coin else None, None
    return None, None, None

In [18]:
# Summarize forecast using only future data
def summarize_forecast(forecast, history_end_date):
    future_forecast = forecast[forecast['ds'] > history_end_date]
    yhat = future_forecast['yhat']
    min_price = yhat.min()
    max_price = yhat.max()
    avg_price = yhat.mean()

    trend = "stable"
    if yhat.iloc[-1] > yhat.iloc[0] * 1.02:
        trend = "upward"
    elif yhat.iloc[-1] < yhat.iloc[0] * 0.98:
        trend = "downward"

    summary = (
        f"The predicted price for the next {len(future_forecast)} days ranges from ${min_price:.2f} "
        f"to ${max_price:.2f}, with an average around ${avg_price:.2f}. "
        f"The overall trend appears to be {trend}."
    )
    return summary

In [19]:
# Handle user commands
def handle_user_input(user_input):
    command, coin, days = parse_user_input(user_input)

    if command == "refresh" and coin:
        full_coin = coin + "-USD"
        try:
            data_store[full_coin] = refresh_data(full_coin)
            return f"Data refreshed for {coin}."
        except Exception as e:
            return f"Error refreshing data for {coin}: {e}"

    elif command == "predict" and coin:
        full_coin = coin + "-USD"
        try:
            if full_coin not in data_store:
                return f"Data for {coin} not found. Please refresh it first."
            forecast, fig = forecast_prices(data_store[full_coin], days)
            forecast_store[full_coin] = forecast
            fig_store[full_coin] = fig
            history_end_date = data_store[full_coin]['ds'].max()
            summary = summarize_forecast(forecast, history_end_date)
            return f"Forecast for {coin} for the next {days} days:\n\n{summary}"
        except Exception as e:
            return f"Error generating prediction for {coin}: {e}"

    elif command == "plot" and coin:
        full_coin = coin + "-USD"
        if full_coin in fig_store:
            fig_store[full_coin].show()
            return f"Here is the forecast plot for {coin}."
        else:
            return f"No forecast found for {coin}. Please run a prediction first."

    else:
        return (
            "Sorry, I didn’t understand. Try saying things like:\n"
            "- 'Refresh BTC'\n"
            "- 'Predict ETH for 10 days'\n"
            "- 'Show BTC chart'"
        )


In [20]:
# Launch Gradio chatbot
gr.Interface(
    fn=handle_user_input,
    inputs="text",
    outputs="text",
    title="🧠 Crypto Forecast Bot",
    description="Ask things like 'predict BTC for 7 days' or 'refresh ETH'"
).launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7033ad23a05cb2938c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
